In [1]:
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from joblib import dump
import seaborn as sns

In [3]:
dat = pd.read_csv("FeatureExtraction/analysis_result_0.01.csv")
#dat.drop(['cc_b', 'cc_g', 'cc_r'], axis=1, inplace=True)
print(dat.head(1))
#dat = dat[dat.edge_diff > 0]
dat = dat.drop_duplicates()

   6.700421512977332  835.7597991132179  101.43943125005683  \
0           7.177873        1448.486609           91.379896   

   0.2072704081632653  43.53509081939833  41.47883756814948  \
0             1.98302          33.495318          33.854145   

   14.986071612452193  -5.714285714285714  0.02146382284869447  0  
0           32.650537           34.839357             0.140486  0  


In [4]:
len(dat)

1151

In [5]:
dat['is_adv'].unique()

KeyError: 'is_adv'

In [6]:
x = dat.iloc[:, :-1].values
x = normalize(x, norm='l2')
y = dat.iloc[:,-1].values

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=1)
#y_train = y_train.reshape([len(y_train), 1])
#y_test = y_test.reshape([len(y_test), 1])

In [8]:
#Remove scaler

sc = StandardScaler()
#x_train = sc.fit_transform(x_train)
#x_test = sc.transform(x_test)
dump(sc, 'Scaler.bin', compress=True)

['Scaler.bin']

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [14]:
keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)

clf = Sequential([
    Dense(512, kernel_initializer='uniform', input_dim=9, activation='elu'),
    Dense(1024, kernel_initializer='uniform', activation='elu'),
    Dropout(0.2),
    Dense(1024, kernel_initializer='uniform', activation='elu'),
    Dense(1, kernel_initializer='uniform', activation='sigmoid')
])

In [15]:
clf.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               5120      
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 1025      
Total params: 1,581,057
Trainable params: 1,581,057
Non-trainable params: 0
_________________________________________________________________


In [16]:
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
clf.fit(x_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
18/18 [==============================] - 1s 12ms/step - loss: 0.6956 - accuracy: 0.5089
Epoch 2/20
18/18 [==============================] - 0s 12ms/step - loss: 0.6342 - accuracy: 0.6554
Epoch 3/20
18/18 [==============================] - 0s 12ms/step - loss: 0.4049 - accuracy: 0.8409

In [18]:
score = clf.evaluate(x_test, y_test, verbose=1)

print(clf.metrics_names)
print(score)

18/18 [==============================] - 0s 3ms/step - loss: 0.0235 - accuracy: 0.9931
['loss', 'accuracy']
[0.023541878908872604, 0.9930555820465088]


In [21]:
from keras.models import load_model
clf.save('AEGuard.keras')

In [20]:
x_predict = [
    [7.3378496842073835,2005.3647338785904,4.504145408163265,32.902195839142415,33.678372772158376,33.41943138869921,75.08849557522123,0.2649477141230245],
    [7.347049941119418,2043.8005363495167,3.0113998724489797,32.896276164881336,33.6823649646088,33.42135887050986,31.436135009927202,0.19496323402345217]
    #[7.109481332039627,1181.620735253842,4.512117346938775,26.801098135999048,37.14988650707845,36.0490153569225,88.47173144876325,0.2652988602177302],
    #[7.145613536455951,1342.1639741820782,3.559470663265306,26.234164188403792,37.466724054725475,36.29911175687073,58.006718924972,0.2217159457752795],
]
#x_predict = sc.transform(x_predict)
y_predict = clf.predict(x_predict)
print(y_predict)

print("Predicted: ", y_predict[0][0], "- adversarial(> 0.5)")
print("Predicted: ", y_predict[1][0], "- original(< 0.5)")

ValueError: in user code:

    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute